# 1. 라이브러리 임포트 및 데이터 불러오기

In [4]:
#라이브러리 임포트
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import scipy.stats as spst
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.model_selection import train_test_split
import haiku as hk
import jax.numpy as jnp

In [2]:
#데이터 불러오기
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

In [3]:
train.head()

,user_id,subscription_duration,recent_login_time,average_login_time,average_time_per_learning_session,monthly_active_learning_days,total_completed_courses,recent_learning_achievement,abandoned_learning_sessions,community_engagement_level,preferred_difficulty_level,subscription_type,customer_inquiry_history,payment_pattern,target
0,b919c29d,13,14,14.946163,8.427187,18,16,68.360455,3,4,Low,Basic,4,5,0
1,a0a60abb,16,18,18.453224,72.646087,16,13,97.567322,2,3,Medium,Basic,1,6,1
2,b9f171ae,22,1,16.195228,21.774492,13,14,94.358763,3,4,Medium,Premium,0,7,1
3,5dc0ba8b,1,19,17.628656,42.659066,19,18,70.153228,0,3,Low,Basic,1,0,1
4,65c83654,4,5,21.390656,30.744287,19,10,81.917908,2,4,Medium,Basic,3,0,1


In [4]:
test.head()

,user_id,subscription_duration,recent_login_time,average_login_time,average_time_per_learning_session,monthly_active_learning_days,total_completed_courses,recent_learning_achievement,abandoned_learning_sessions,community_engagement_level,preferred_difficulty_level,subscription_type,customer_inquiry_history,payment_pattern
0,0001d6e9,9,15,11.173051,7.047243,19,15,82.074557,3,5,Low,Premium,2,7
1,0002c77d,11,8,13.448592,24.583327,10,16,80.546040,4,2,Low,Basic,4,4
2,0002df5b,11,28,18.548201,1.707616,9,19,73.676957,5,5,Low,Premium,1,3
3,000b6068,14,17,13.383311,21.391389,12,6,63.314635,0,4,High,Premium,1,5
4,00184a0c,19,6,11.919663,18.126987,8,15,55.899647,1,4,Medium,Basic,1,3


# 2. 데이터 분석 및 전처리

In [5]:
#데이터 결측치 확인
print(train.isnull().sum())
print(test.isnull().sum())

user_id                              0
subscription_duration                0
recent_login_time                    0
average_login_time                   0
average_time_per_learning_session    0
monthly_active_learning_days         0
total_completed_courses              0
recent_learning_achievement          0
abandoned_learning_sessions          0
community_engagement_level           0
preferred_difficulty_level           0
subscription_type                    0
customer_inquiry_history             0
payment_pattern                      0
target                               0
dtype: int64
user_id                              0
subscription_duration                0
recent_login_time                    0
average_login_time                   0
average_time_per_learning_session    0
monthly_active_learning_days         0
total_completed_courses              0
recent_learning_achievement          0
abandoned_learning_sessions          0
community_engagement_level           0
preferred_di

In [6]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 15 columns):
 #   Column                             Non-Null Count  Dtype  
---  ------                             --------------  -----  
 0   user_id                            10000 non-null  object 
 1   subscription_duration              10000 non-null  int64  
 2   recent_login_time                  10000 non-null  int64  
 3   average_login_time                 10000 non-null  float64
 4   average_time_per_learning_session  10000 non-null  float64
 5   monthly_active_learning_days       10000 non-null  int64  
 6   total_completed_courses            10000 non-null  int64  
 7   recent_learning_achievement        10000 non-null  float64
 8   abandoned_learning_sessions        10000 non-null  int64  
 9   community_engagement_level         10000 non-null  int64  
 10  preferred_difficulty_level         10000 non-null  object 
 11  subscription_type                  10000 non-null  obje

In [7]:
#기초 통계 확인

pd.set_option('display.max_columns', None) #모든 값 출력
train.describe()

,subscription_duration,recent_login_time,average_login_time,average_time_per_learning_session,monthly_active_learning_days,total_completed_courses,recent_learning_achievement,abandoned_learning_sessions,community_engagement_level,customer_inquiry_history,payment_pattern,target
count,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000
mean,11.897400,15.013200,14.994076,54.917720,12.545400,12.227500,75.029513,3.043600,3.886100,2.010500,3.502900,0.619900
std,6.600896,8.362573,3.001869,56.024310,6.932239,3.634125,9.968529,1.755052,1.262175,1.420983,2.311261,0.485435
min,1.000000,1.000000,2.366189,0.011515,1.000000,1.000000,35.941755,0.000000,1.000000,0.000000,0.000000,0.000000
25%,6.000000,8.000000,13.025597,15.276611,7.000000,10.000000,68.278054,2.000000,3.000000,1.000000,1.000000,0.000000
50%,12.000000,15.000000,14.979228,37.578818,13.000000,12.000000,75.126061,3.000000,4.000000,2.000000,4.000000,1.000000
75%,18.000000,22.000000,16.995340,75.584200,19.000000,15.000000,81.718976,4.000000,5.000000,3.000000,6.000000,1.000000
max,23.000000,29.000000,26.998490,503.372616,24.000000,27.000000,112.643828,12.000000,5.000000,10.000000,7.000000,1.000000


In [8]:
# y값의 비율 보기

train['target'].value_counts()

1    6199
0    3801
Name: target, dtype: int64

In [9]:
from sklearn.preprocessing import LabelEncoder

# 라벨인코딩을 적용할 변수의 리스트를 생성합니다.
features = ['preferred_difficulty_level', 'subscription_type']

# 원하는 값을 지정할 딕셔너리를 생성합니다.
mapping_1 = {
    'Low': 0,
    'Medium': 1,
    'High': 2
}

mapping_2 = {
    'Basic': 0,
    'Premium': 1,
}



le = LabelEncoder()
# 라벨인코딩 클래스에 매핑 딕셔너리를 적용합니다.
le.fit(train['preferred_difficulty_level'])
train['preferred_difficulty_level'] = train['preferred_difficulty_level'].map(mapping_1)

le.fit(train['subscription_type'])
train['subscription_type'] = train['subscription_type'].map(mapping_2)

# test 데이터도 똑같은 처리
le.fit(test['preferred_difficulty_level'])
test['preferred_difficulty_level'] = test['preferred_difficulty_level'].map(mapping_1)

le.fit(test['subscription_type'])
test['subscription_type'] = test['subscription_type'].map(mapping_2)



In [10]:
#x, y 분리

x_train = train.drop(columns=['target','user_id'], axis=1)
test_id = test['user_id']
test = test.drop('user_id', axis=1)
y_train = train['target']

In [11]:
numeric_train = x_train.select_dtypes(include=['int', 'float'])

# VIF 계산
vif = pd.DataFrame()
vif["VIF Factor"] = [variance_inflation_factor(numeric_train.values, i) for i in range(numeric_train.shape[1])]
vif["features"] = numeric_train.columns
vif = vif.sort_values(by="VIF Factor", ascending=False)

print(vif)

    VIF Factor                           features
6    31.978938        recent_learning_achievement
2    20.644732                 average_login_time
5    14.320447            total_completed_courses
8    12.883978         community_engagement_level
4     4.196620       monthly_active_learning_days
0     4.150712              subscription_duration
1     4.129185                  recent_login_time
7     3.930994        abandoned_learning_sessions
12    3.246296                    payment_pattern
11    2.962913           customer_inquiry_history
10    2.404611                  subscription_type
3     1.999792  average_time_per_learning_session
9     1.952438         preferred_difficulty_level


# 3. 모델 

In [12]:
x_train.shape , y_train.shape

((10000, 13), (10000,))

In [13]:
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size= 0.3 , random_state=42)

In [14]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()

x_train = scaler.fit_transform(x_train)
x_val = scaler.fit_transform(x_val)
test = scaler.fit_transform(test)

In [15]:
from imblearn.over_sampling import SMOTE

smote = SMOTE()
X_resampled, y_resampled = smote.fit_resample(x_train, y_train)

print(np.bincount(y_train))
print(np.bincount(y_train) / y_train.shape[0])

[2659 4341]
[0.37985714 0.62014286]


In [16]:
y_resampled.value_counts()

1    4341
0    4341
Name: target, dtype: int64

In [17]:
import tensorflow.keras.backend as K

# Macro F1 스코어 계산 함수
def macro_f1_score(y_true, y_pred):
    def recall(y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = true_positives / (possible_positives + K.epsilon())
        return recall

    def precision(y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision

    precision = precision(y_true, y_pred)
    recall = recall(y_true, y_pred)
    f1_score = 2 * ((precision * recall) / (precision + recall + K.epsilon()))
    return f1_score

In [18]:
#############
# Your Code # Sequential API
#############
# 1번 세션 클리어
keras.backend.clear_session()
# 2번 모델 선언
model = keras.models.Sequential()
# 3번 모델 조립
model.add(keras.layers.Input(shape=(13,)) )
model.add(keras.layers.Dense(128, activation='relu') )
model.add(keras.layers.Dropout(0.25))
model.add(keras.layers.Dense(64, activation='relu') )
model.add(keras.layers.Dense(32, activation='relu') )
model.add(keras.layers.Dense(1, activation='sigmoid') )
# 4번 컴파일

# 모델 컴파일 시 class_weight 매개변수를 설정하여 가중치 적용
model.compile(loss='binary_crossentropy', metrics=[macro_f1_score],
              optimizer='adam')
# 요약
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 128)               1792      
                                                                 
 dropout (Dropout)           (None, 128)               0         
                                                                 
 dense_1 (Dense)             (None, 64)                8256      
                                                                 
 dense_2 (Dense)             (None, 32)                2080      
                                                                 
 dense_3 (Dense)             (None, 1)                 33        
                                                                 
Total params: 12,161
Trainable params: 12,161
Non-trainable params: 0
_________________________________________________________________


In [19]:
from keras.callbacks import EarlyStopping
es = EarlyStopping(monitor='val_macro_f1_score',patience=30,min_delta=0,restore_best_weights=True,verbose=1)

In [20]:
history = model.fit(X_resampled, y_resampled, epochs=10000, verbose=1,batch_size=16, validation_data=(x_val,y_val),callbacks=[es])

Epoch 1/10000
543/543 [==============================] - 3s 3ms/step - loss: 0.6944 - macro_f1_score: 0.4383 - val_loss: 0.6866 - val_macro_f1_score: 0.6684
Epoch 2/10000
543/543 [==============================] - 2s 3ms/step - loss: 0.6897 - macro_f1_score: 0.4538 - val_loss: 0.6803 - val_macro_f1_score: 0.4758
Epoch 3/10000
543/543 [==============================] - 2s 3ms/step - loss: 0.6804 - macro_f1_score: 0.3155 - val_loss: 0.6785 - val_macro_f1_score: 0.2298
Epoch 4/10000
543/543 [==============================] - 2s 3ms/step - loss: 0.6767 - macro_f1_score: 0.3229 - val_loss: 0.6785 - val_macro_f1_score: 0.3144
Epoch 5/10000
543/543 [==============================] - 2s 3ms/step - loss: 0.6736 - macro_f1_score: 0.2748 - val_loss: 0.6736 - val_macro_f1_score: 0.3070
Epoch 6/10000
543/543 [==============================] - 2s 3ms/step - loss: 0.6724 - macro_f1_score: 0.2934 - val_loss: 0.6790 - val_macro_f1_score: 0.3688
Epoch 7/10000
543/543 [==============================] - 2

In [21]:
y_pred = model.predict(test)

In [22]:
y_pred

array([[0.4208435 ],
       [0.44856924],
       [0.4040758 ],
       ...,
       [0.46888715],
       [0.3949163 ],
       [0.44431576]], dtype=float32)

In [23]:
threshold = 0.5  # 임계값 설정

# y_pred에서 확률 값에 따라 0 또는 1로 이진 분류
y_pred_binary = np.where(y_pred >= threshold, 1, 0)
y_pred_binary

array([[0],
       [0],
       [0],
       ...,
       [0],
       [0],
       [0]])

In [24]:
# 예시 데이터프레임 생성 (user_id 열만 있는 데이터프레임)
df1 = pd.DataFrame(test_id)
# 예측값 배열 (임의의 값으로 예시 작성)

flattened_y_pred = y_pred_binary.flatten()

# 'user_id'와 'target' 열을 가진 데이터프레임 생성
df2 = pd.DataFrame({
    'user_id': df1['user_id'],
    'target': flattened_y_pred
})

# 새로운 데이터프레임 출력
print(df2)

       user_id  target
0     0001d6e9       0
1     0002c77d       0
2     0002df5b       0
3     000b6068       0
4     00184a0c       0
...        ...     ...
9995  ffe2eba5       0
9996  ffe710f1       0
9997  ffeccdef       0
9998  fff3fcea       0
9999  fff4b04b       0

[10000 rows x 2 columns]


In [25]:
df2.to_csv('result.csv', index=False)